# [1단계] 환경 설정

## 1-1. 필요한 라이브러리 설치 및 임포트

In [1]:
!pip install -qU transformers datasets "pandas==2.2.2" huggingface_hub

In [2]:
from datasets import load_dataset, concatenate_datasets, DatasetDict
from transformers import AutoTokenizer
from huggingface_hub import login
from google.colab import userdata

print("✅ 라이브러리 준비 완료!")

✅ 라이브러리 준비 완료!


## 1-2. Hugging Face 로그인

In [3]:
try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    print("✅ Hugging Face 로그인 성공!")
except Exception as e:
    print(f"🚨 Hugging Face 로그인 실패: {e}")

✅ Hugging Face 로그인 성공!


# [2단계] 3종 데이터셋 로드 (수정된 소스)

In [4]:
print("--- 3종 데이터셋 로드를 시작합니다 (beomi/KoAlpaca-RealQA 포함) ---")

try:
    hf_username = userdata.get('HF_USERNAME')

    # 1. KB 특화 지식 데이터셋 (우리가 직접 생성하고 정제한 버전)
    # train과 test 스플릿을 모두 합쳐서 사용할 것이므로, 두 스플릿을 모두 로드한다.
    kb_train = load_dataset(f"{hf_username}/KB-sLLM-QA-Dataset-Final-Split", split="train")
    kb_test = load_dataset(f"{hf_username}/KB-sLLM-QA-Dataset-Final-Split", split="test")
    kb_dataset = concatenate_datasets([kb_train, kb_test])
    print(f"✅ 1. KB 특화 데이터셋 로드 완료: {len(kb_dataset)}건")

    # 2. 일반 금융 지식 데이터셋
    fin_dataset = load_dataset("aiqwe/FinShibainu", "qa", split="train")
    print(f"✅ 2. 일반 금융 데이터셋 로드 완료: {len(fin_dataset)}건")

    # 3. (수정) 고품질 일상 대화 데이터셋
    common_dataset = load_dataset("beomi/KoAlpaca-RealQA", split="train")
    print(f"✅ 3. (수정) 일상 대화 데이터셋 로드 완료: {len(common_dataset)}건")

    print("\n\n--- 각 데이터셋의 구조(features) 확인 ---")
    print("\n[KB 특화 데이터셋 구조]")
    print(kb_dataset.features)
    print("\n[일반 금융 데이터셋 구조]")
    print(fin_dataset.features)
    print("\n[일상 대화(KoAlpaca) 데이터셋 구조]")
    print(common_dataset.features)

except Exception as e:
    print(f"🚨 데이터셋 로드 중 오류가 발생했습니다: {e}")

--- 3종 데이터셋 로드를 시작합니다 (beomi/KoAlpaca-RealQA 포함) ---


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/197k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1007 [00:00<?, ? examples/s]

...

✅ 2. 일반 금융 데이터셋 로드 완료: 44870건


README.md:   0%|          | 0.00/7.17k [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/14.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18524 [00:00<?, ? examples/s]

✅ 3. (수정) 일상 대화 데이터셋 로드 완료: 18524건


--- 각 데이터셋의 구조(features) 확인 ---

[KB 특화 데이터셋 구조]
{'question': Value('string'), 'answer_B': Value('string'), 'source_pdf': Value('string')}

[일반 금융 데이터셋 구조]
{'reference': Value('string'), 'question': Value('string'), 'answer_A': Value('string'), 'answer_B': Value('string'), 'preference': Value('string'), 'preference_desc': Value('string'), 'value': Value('int64'), 'type': Value('string')}

[일상 대화(KoAlpaca) 데이터셋 구조]
{'custom_id': Value('string'), 'question': Value('string'), 'answer': Value('string')}


# [3단계] 데이터셋 형식 통일 (수정된 소스)

## 3-1. 'question'/'answer_B' 형식을 변환하는 함수 (KB, 금융용)

In [5]:
def format_qa_to_conversations(example):
    # 'answer_B' 컬럼이 없는 경우를 대비해 유연하게 처리
    answer = example.get("answer_B") or example.get("answer")
    return {
        "conversations": [
            {"role": "user", "content": example["question"]},
            {"role": "assistant", "content": answer}
        ]
    }

## 3-2. 'KoAlpaca' 형식을 변환하는 함수

In [6]:
# 위 format_qa_to_conversations 함수가 'answer' 컬럼도 처리할 수 있으므로,
# 별도의 함수 없이 재사용할 수 있다. 구조가 거의 동일하기 때문!

print("--- 데이터셋 형식 통일을 시작합니다 ---")

# KB 특화 데이터셋에 함수 적용
kb_dataset_formatted = kb_dataset.map(
    format_qa_to_conversations,
    remove_columns=kb_dataset.column_names
)
print("✅ 1. KB 특화 데이터셋 형식 통일 완료!")

# 일반 금융 데이터셋에 함수 적용
fin_dataset_formatted = fin_dataset.map(
    format_qa_to_conversations,
    remove_columns=fin_dataset.column_names
)
print("✅ 2. 일반 금융 데이터셋 형식 통일 완료!")

# KoAlpaca 데이터셋에 함수 적용
common_dataset_formatted = common_dataset.map(
    format_qa_to_conversations,
    remove_columns=common_dataset.column_names
)
print("✅ 3. (수정) KoAlpaca 데이터셋 형식 통일 완료!")


print("\n\n--- 형식 통일 후 샘플 데이터 확인 ---")
print("\n[KB 특화 데이터셋 샘플]")
print(kb_dataset_formatted[0]['conversations'])
print("\n[일반 금융 데이터셋 샘플]")
print(fin_dataset_formatted[0]['conversations'])
print("\n[KoAlpaca 데이터셋 샘플]")
print(common_dataset_formatted[0]['conversations'])

--- 데이터셋 형식 통일을 시작합니다 ---


Map:   0%|          | 0/1157 [00:00<?, ? examples/s]

✅ 1. KB 특화 데이터셋 형식 통일 완료!


Map:   0%|          | 0/44870 [00:00<?, ? examples/s]

✅ 2. 일반 금융 데이터셋 형식 통일 완료!


Map:   0%|          | 0/18524 [00:00<?, ? examples/s]

✅ 3. (수정) KoAlpaca 데이터셋 형식 통일 완료!


--- 형식 통일 후 샘플 데이터 확인 ---

[KB 특화 데이터셋 샘플]
[{'content': '원리금균등 분할상환과 원금균등 분할상환의 차이점은 무엇인가요?', 'role': 'user'}, {'content': '원리금균등 분할상환은 매월 이자지급일에 동일한 할부금을 상환하는 방식이며, 원금균등 분할상환은 매월 이자지급일에 이자와 동일한 할부 원금을 상환하는 방식입니다.', 'role': 'assistant'}]

[일반 금융 데이터셋 샘플]
[{'content': '가계부문의 순저축률을 측정하는 공식을 적고, 각 항목이 의미하는 바를 설명하시오.', 'role': 'user'}, {'content': '가계부문의 순저축률은 가계가 얼마나 저축하고 있는지를 나타내는 중요한 경제 지표로, 일반적으로 다음과 같은 공식을 사용하여 측정합니다:\n\n\\[ 순저축률 = \\frac{순저축}{가처분소득} \\times 100 \\]  \n\n### 각 항목의 의미\n\n1. **순저축 (Net Savings)**:  \n   - 순저축은 가계가 일정 기간(예: 1년) 동안 얻은 소득에서 소비지출을 제한 후 남는 금액을 의미합니다. \n   - 공식적으로는 다음과 같이 표현됩니다:  \n   \\[ 순저축 = 가처분소득 - 소비지출 \\]  \n   - 여기서 "가처분소득"은 세금을 낸 후 실제로 사용할 수 있는 소득을 의미하며, "소비지출"은 생활비, 세금, 보험료 등 가계에서 지출한 모든 비용을 포함합니다.\n\n2. **가처분소득 (Disposable Income)**:  \n   - 가처분소득은 가계가 세금 지급 후 사용할 수 있는 모든 소득을 의미합니다.  \n   - 이는 노동 소득(임금, 급여), 자산 소득(이자, 배당금) 등 모든 형태의 소득에서 세금을 제외한 것을 의미합니다.\n   - 가처분소득은 가계의 재정적 여유를 측정하는 중요한 지표로 사용됩니다.\n\n3. **순저축률 (S

# [4단계] 데이터셋 샘플링 및 통합 (beomi/KoAlpaca-RealQA 기반)

In [7]:
# 각 데이터셋에서 사용할 개수를 정의. (일상 15k, 금융 15k, KB 전체)
N_COMMON = 15000 # 일상 대화 데이터 (KoAlpaca)
N_FIN = 15000    # 일반 금융 데이터 (FinShibainu)

print(f"--- 데이터셋 샘플링 및 통합을 시작합니다 ---")
print(f"사용할 데이터 개수: KB특화({len(kb_dataset_formatted)}건), 일반금융({N_FIN}건), 일상대화({N_COMMON}건)")

# 데이터셋을 섞은 후, 필요한 개수만큼만 선택(샘플링).
# seed=42를 설정해서 항상 동일한 순서로 섞이도록 보장.
common_sampled = common_dataset_formatted.shuffle(seed=42).select(range(N_COMMON))
fin_sampled = fin_dataset_formatted.shuffle(seed=42).select(range(N_FIN))

print("✅ 일반 금융 및 일상 대화 데이터셋 샘플링 완료!")

# 3개의 데이터셋을 하나의 리스트에 담아 합친다.
final_dataset = concatenate_datasets([
    kb_dataset_formatted, # KB 데이터는 전부 사용
    fin_sampled,
    common_sampled
])

# 합쳐진 전체 데이터셋을 다시 한번 섞어준다.
final_dataset_shuffled = final_dataset.shuffle(seed=42)

total_count = len(final_dataset_shuffled)
print(f"\n✅ 3종 데이터셋 통합 및 셔플 완료!")
print(f"최종 데이터셋 크기: {total_count}건")

# 통합된 데이터셋의 샘플을 확인.
print("\n--- 통합된 데이터셋 샘플 ---")
print(final_dataset_shuffled[0]['conversations'])

--- 데이터셋 샘플링 및 통합을 시작합니다 ---
사용할 데이터 개수: KB특화(1157건), 일반금융(15000건), 일상대화(15000건)
✅ 일반 금융 및 일상 대화 데이터셋 샘플링 완료!

✅ 3종 데이터셋 통합 및 셔플 완료!
최종 데이터셋 크기: 31157건

--- 통합된 데이터셋 샘플 ---
[{'content': '주식집단의 매도와 동시에 매수하는 선물거래의 의미는 무엇인가요?', 'role': 'user'}, {'content': '주식집단의 매도와 동시에 매수하는 선물거래는 주식 시장에서의 헤지(hedging) 전략 또는 차익 거래(arbitrage) 전략으로 이해할 수 있습니다. 이러한 거래는 특정 시장 조건이나 투자 전략에 따라 다양하게 활용되며, 아래와 같은 주요 개념을 포함합니다.\n\n### 1. 선물 거래란?\n선물 거래는 특정 자산을 미래의 특정 시점에 미리 정해진 가격으로 거래하기로 약속하는 계약입니다. 이러한 계약은 주식, 채권, 원자재 등 다양한 자산에 적용될 수 있으며, 투자자들은 이를 통해 가격 변동에 대한 리스크를 관리하거나, 가격 차이를 이용해 이익을 낼 수 있습니다.\n\n### 2. 주식집단의 매도와 매수\n- **매도(Short Selling)**: 투자자가 특정 주식을 빌려서 팔고, 나중에 가격이 하락했을 때 다시 사서 갚는 방식입니다. 이는 주가 하락에 베팅하는 전략으로, 가격이 하락하면 이익을 얻을 수 있습니다.  \n- **매수(Long Position)**: 반대로, 주식을 사는 것입니다. 주식의 가격이 상승할 것으로 예상하여 주식을 사는 전략입니다.\n\n### 3. 동시에 매도와 매수하는 이유\n주식집단을 한꺼번에 매도하고 매수하는 이유는 다양합니다.\n- **헤지 전략**: 예를 들어, 특정 주식의 가격이 하락할 것으로 예상되는 경우, 해당 주식을 매도하면서 동시에 선물 계약을 매수하여 가격 하락의 위험을 줄일 수 있습니다. 즉, 주식의 손실을 선물에서의 이익으로 상쇄하려는 의도입니다.\n- *

# [5단계] Qwen2 Instruct 템플릿 적용 및 최종 데이터셋 업로드

## 5-1. Qwen2 Instruct 토크나이저 로드

In [8]:
# 토론의 결론에 따라, Instruct 모델의 공식 토크나이저를 직접 사용한다.
# 이렇게 하면 EOS 토큰과 챗 템플릿 관련 문제를 모두 해결할 수 있다.
instruct_tokenizer_id = "Qwen/Qwen2.5-14B-Instruct"
try:
    tokenizer = AutoTokenizer.from_pretrained(instruct_tokenizer_id)
    print(f"✅ '{instruct_tokenizer_id}' 토크나이저 로드 성공!")
except Exception as e:
    print(f"🚨 토크나이저 로드 실패: {e}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ 'Qwen/Qwen2.5-14B-Instruct' 토크나이저 로드 성공!


## 5-2. 데이터셋에 Qwen2 챗 템플릿 적용

In [9]:
def apply_chat_template(example):
    """
    대화(conversations) 리스트를 토크나이저를 사용해 단일 텍스트로 변환한다.
    """
    return {"text": tokenizer.apply_chat_template(
        example["conversations"], tokenize=False, add_generation_prompt=False
    )}

print("\n--- Qwen2 챗 템플릿 적용을 시작합니다 ---")
final_dataset_templated = final_dataset_shuffled.map(
    apply_chat_template,
    remove_columns=["conversations"]
)
print("✅ 템플릿 적용 완료!")
print("\n--- 템플릿 적용 후 샘플 데이터 ---")
print(final_dataset_templated[0]['text'])


--- Qwen2 챗 템플릿 적용을 시작합니다 ---


Map:   0%|          | 0/31157 [00:00<?, ? examples/s]

✅ 템플릿 적용 완료!

--- 템플릿 적용 후 샘플 데이터 ---
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
주식집단의 매도와 동시에 매수하는 선물거래의 의미는 무엇인가요?<|im_end|>
<|im_start|>assistant
주식집단의 매도와 동시에 매수하는 선물거래는 주식 시장에서의 헤지(hedging) 전략 또는 차익 거래(arbitrage) 전략으로 이해할 수 있습니다. 이러한 거래는 특정 시장 조건이나 투자 전략에 따라 다양하게 활용되며, 아래와 같은 주요 개념을 포함합니다.

### 1. 선물 거래란?
선물 거래는 특정 자산을 미래의 특정 시점에 미리 정해진 가격으로 거래하기로 약속하는 계약입니다. 이러한 계약은 주식, 채권, 원자재 등 다양한 자산에 적용될 수 있으며, 투자자들은 이를 통해 가격 변동에 대한 리스크를 관리하거나, 가격 차이를 이용해 이익을 낼 수 있습니다.

### 2. 주식집단의 매도와 매수
- **매도(Short Selling)**: 투자자가 특정 주식을 빌려서 팔고, 나중에 가격이 하락했을 때 다시 사서 갚는 방식입니다. 이는 주가 하락에 베팅하는 전략으로, 가격이 하락하면 이익을 얻을 수 있습니다.  
- **매수(Long Position)**: 반대로, 주식을 사는 것입니다. 주식의 가격이 상승할 것으로 예상하여 주식을 사는 전략입니다.

### 3. 동시에 매도와 매수하는 이유
주식집단을 한꺼번에 매도하고 매수하는 이유는 다양합니다.
- **헤지 전략**: 예를 들어, 특정 주식의 가격이 하락할 것으로 예상되는 경우, 해당 주식을 매도하면서 동시에 선물 계약을 매수하여 가격 하락의 위험을 줄일 수 있습니다. 즉, 주식의 손실을 선물에서의 이익으로 상쇄하려는 의도입니다.
- **차익 거래 기회**: 시장에서의 가격 차이를 이용하여 수익을 추구하는 전략입니다.

## 5-3. 학습용 및 평가용 데이터셋으로 분할

In [10]:
split_dataset = final_dataset_templated.train_test_split(test_size=0.05, seed=42)

print("\n--- 데이터셋 분할 완료 ---")
print(f"훈련 데이터셋: {len(split_dataset['train'])}건")
print(f"평가 데이터셋: {len(split_dataset['test'])}건")


--- 데이터셋 분할 완료 ---
훈련 데이터셋: 29599건
평가 데이터셋: 1558건


## 5-4. 최종 데이터셋을 Hugging Face Hub에 업로드

In [11]:
try:
    hf_username = userdata.get('HF_USERNAME')
    repo_name = f"{hf_username}/combined-dataset-30K-final-v2" # v2로 업로드

    print(f"\n🚀 최종 데이터셋을 Hugging Face Hub에 업로드합니다...")
    print(f"Repository: {repo_name}")

    split_dataset.push_to_hub(
        repo_id=repo_name,
        private=True,
        commit_message="Upload v2 dataset (KB + FinShibainu + KoAlpaca-RealQA)"
    )

    print("\n🎉 최종 데이터셋 업로드 성공!")
    print(f"URL: https://huggingface.co/datasets/{repo_name}")

except Exception as e:
    print(f"🚨 데이터셋 업로드 중 오류가 발생했습니다: {e}")


🚀 최종 데이터셋을 Hugging Face Hub에 업로드합니다...
Repository: rucipheryn/combined-dataset-30K-final-v2


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/30 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]


🎉 최종 데이터셋 업로드 성공!
URL: https://huggingface.co/datasets/rucipheryn/combined-dataset-30K-final-v2
